In [3]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
Attack_list = ['Bruteforce_CWE-307','CVE-2012-2122','CVE-2014-0160','CVE-2017-7529','CVE-2018-3760',
               'CVE-2019-5418','EPS_CWE-434','PHP_CWE-434','SQL_Injection_CWE-89','ZipSlip']
Attack_list

['Bruteforce_CWE-307',
 'CVE-2012-2122',
 'CVE-2014-0160',
 'CVE-2017-7529',
 'CVE-2018-3760',
 'CVE-2019-5418',
 'EPS_CWE-434',
 'PHP_CWE-434',
 'SQL_Injection_CWE-89',
 'ZipSlip']

In [5]:
def make_csv(name): # .txt -> .csv converting function
    file_list = os.listdir("/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/"+name)
    file_list.remove('.DS_Store') #Mac의 경우만 존재하는 것 같음 Mac환경이 아니라면 이 row를 무방함
    print(len(file_list))
    base_dir = '/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/'+name+'/'#txt파일이 들어있는 폴더가 있는 dir
    base_csv_dir = '/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/'#CSV파일을 저장할 폴더
    for e in tqdm(file_list):
        dir = base_dir + e
        sample = pd.read_table(dir, header=None)
        tmp = list()
        for i in range (sample.shape[0]):
            x = sample[0][i]
            sp = x.split(' ', 8)
            tmp.append(sp)
        data = pd.DataFrame(tmp,columns=['event_num','event_time','cpu','user_uid','process','process_id','event_direction','event_type','event_argument'])
        data.reset_index(drop=True,inplace=True)
        file_name_csv = e.replace('.txt','.csv')
        save_dir = base_csv_dir + file_name_csv
        data.to_csv(save_dir)

In [241]:
for e in Attack_list: #.txt -> .csv converting part
    print(e)
    make_csv(e)

1221


100%|███████████████████████████████████████| 1221/1221 [04:23<00:00,  4.64it/s]


1079


100%|███████████████████████████████████████| 1079/1079 [04:04<00:00,  4.41it/s]


1071


...

100%|███████████████████████████████████████| 1112/1112 [05:47<00:00,  3.20it/s]


1078


100%|███████████████████████████████████████| 1078/1078 [06:06<00:00,  2.94it/s]


1063


100%|█████████████████████████████████████| 1063/1063 [1:07:01<00:00,  3.78s/it]


In [6]:
def runs_preprocessing(name): #run_file preprocessing function
    run_file = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/run/runs.csv')
    #runs.csv파일을 저는 CSV/Bruteforce_CWE-307/run/runs.csv와 같은 dir에 넣어놓았습니다.
    #run_file에 해당공격의 runs.csv파일을 읽어오게 하시면 됩니다.
    run_file.columns = ['image_name','scenario_name','is_executing_exploit','warmup_time',
                       'recording_time', 'exploit_start_time']
    run_file['image_name'] = run_file['image_name'].str.strip()
    run_file['scenario_name'] = run_file['scenario_name'].str.strip()
    run_file.to_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/run/runs.csv', index=False)
    #이부분도 원하시는 dir를 설정하여 저장하시면 됩니다.

In [526]:
for e in tqdm(Attack_list): # run_file preprocessing pa
    print(e)
    runs_preprocessing(e)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 159.70it/s]

Bruteforce_CWE-307
CVE-2012-2122
CVE-2014-0160
CVE-2017-7529
CVE-2018-3760
CVE-2019-5418
EPS_CWE-434
PHP_CWE-434
SQL_Injection_CWE-89
ZipSlip


In [65]:
def data_preprocessing(name): # data_preprocessing(labeling,...)
    file_list = os.listdir("/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/"+name)
    #CSV파일 리스트 읽어오기
    file_list.remove('.DS_Store') #Mac의 경우만 존재하는 것 같음
    file_list.remove('run') #저는 /CSV/run 폴더에 runs.csv파일을 넣어놓았기 때문에 디렉토리를 삭제
    print(len(file_list))
    base_csv_dir = '/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/'
    #CSV파일이 들어있는 폴더
    for e in tqdm(file_list):
        dir = base_csv_dir + e
        n = e.replace('.csv','')
        sample = pd.read_csv(dir, index_col=0)
        sample['event_time'] = sample['event_time'].str.slice(start=0, stop=8)
        sample['label'] = 0
        run_file = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/run/runs.csv')
        #해당 공격의 runs.csv파일을 읽어오게 하면 됩니다. dir 설정부탁
        if run_file['exploit_start_time'][run_file['scenario_name']==n].values[0] != -1:
            record_start = int(sample['event_time'][0][6:8])#time split only second
            exploit_start = run_file['exploit_start_time'][run_file['scenario_name']==n].values[0]
            tmp_time = record_start + exploit_start - 1 #modified
            if tmp_time > 60:
                real_exploit_sec = tmp_time - 60  
            else:
                real_exploit_sec = tmp_time  
            sample.loc[sample['event_time'].str.slice(6,8) == str(real_exploit_sec),'label'] = 1
        sample.to_csv(dir)
            #그대로 저장

In [539]:
for e in Attack_list:
    print(e)
    data_preprocessing(e)

Bruteforce_CWE-307
1092


100%|███████████████████████████████████████| 1092/1092 [00:39<00:00, 27.57it/s]


CVE-2012-2122
1395


100%|███████████████████████████████████████| 1395/1395 [00:45<00:00, 30.47it/s]


CVE-2014-0160
1100


...

100%|███████████████████████████████████████| 1112/1112 [07:35<00:00,  2.44it/s]


SQL_Injection_CWE-89
1078


100%|███████████████████████████████████████| 1078/1078 [03:44<00:00,  4.80it/s]


ZipSlip
1063


100%|███████████████████████████████████████| 1063/1063 [28:22<00:00,  1.60s/it]


---------------------------------------------------------------------------------------------------------------
EDA Part

In [48]:
def unique_function(name):#event_type, process 종류 추출 함수
    file_list = os.listdir("/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/"+name)
    file_list.remove('.DS_Store') #Mac의 경우만 존재하는 것 같음
    file_list.remove('run') #저는 /CSV/run 폴더에 runs.csv파일을 넣어놓았기 때문에 디렉토리를 삭제
    process_values = np.array([])
    event_type_values = np.array([])
    print(len(file_list))
    for e in tqdm(file_list):
        dir = "/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/"+name+"/"+e
        sample = pd.read_csv(dir, index_col=0)
        tmp1 = sample['process'].unique()
        tmp2 = sample['event_type'].unique()
        process_values = np.append(process_values,tmp1)
        event_type_values = np.append(event_type_values,tmp2)
    process_values = process_values.astype(str)
    process_values = np.unique(process_values)
    event_type_values = event_type_values.astype(str)
    event_type_values = np.unique(event_type_values)
    print(process_values)
    print(event_type_values)
    return process_values, event_type_values

In [49]:
process_arr = np.array([])
event_type_arr = np.array([])
for e in Attack_list:
    print(e + " EDA part")
    process_values, event_type_values = unique_function(e)
    process_arr = np.append(process_arr,process_values)
    event_type_arr = np.append(event_type_arr, event_type_values)
    print("process list :")
    print(process_arr)
    print("event type list : ")
    print(event_type_arr)
process_arr = np.unique(process_arr) #process 종류를 담은 배열
event_type_arr = np.unique(event_type_arr)#event_type을 담은 배열
print(process_arr)
print(event_type_arr)

Bruteforce_CWE-307 EDA part
1092


100%|███████████████████████████████████████| 1092/1092 [00:11<00:00, 94.37it/s]


['apache2' 'nan']
['accept' 'brk' 'chdir' 'chmod' 'clone' 'close' 'connect' 'epoll_create1'
 'epoll_ctl' 'epoll_wait' 'exit_group' 'fcntl' 'fstat' 'getcwd' 'getdents'
 'geteuid' 'getsockname' 'getsockopt' 'lseek' 'lstat' 'mmap' 'munmap'
 'open' 'openat' 'poll' 'procexit' 'read' 'rename' 'rt_sigaction'
 'rt_sigprocmask' 'select' 'semop' 'set_robust_list' 'setgid' 'setgroups'
 'setitimer' 'setuid' 'shutdown' 'socket' 'stat' 'switch' 'times' 'umask'
 'wait4' 'write' 'writev']
process list :
['apache2' 'nan']
event type list : 
['accept' 'brk' 'chdir' 'chmod' 'clone' 'close' 'connect' 'epoll_create1'
 'epoll_ctl' 'epoll_wait' 'exit_group' 'fcntl' 'fstat' 'getcwd' 'getdents'
 'geteuid' 'getsockname' 'getsockopt' 'lseek' 'lstat' 'mmap' 'munmap'
 'open' 'openat' 'poll' 'procexit' 'read' 'rename' 'rt_sigaction'
 'rt_sigprocmask' 'select' 'semop' 'set_robust_list' 'setgid' 'setgroups'
 'setitimer' 'setuid' 'shutdown' 'socket' 'stat' 'switch' 'times' 'umask'
 'wait4' 'write' 'writev']
CVE-2012-2

100%|██████████████████████████████████████| 1395/1395 [00:12<00:00, 114.94it/s]


['mysqld' 'mysqld_safe' 'nan']
['accept' 'access' 'clone' 'close' 'exit' 'fcntl' 'fsync' 'futex'
 'getpeername' 'getsockname' 'madvise' 'mmap' 'mprotect' 'munmap' 'poll'
 'procexit' 'pwrite' 'read' 'sched_yield' 'select' 'set_robust_list'
 'setsockopt' 'shutdown' 'stat' 'switch' 'wait4' 'write']
process list :
['apache2' 'nan' 'mysqld' 'mysqld_safe' 'nan']
event type list : 
['accept' 'brk' 'chdir' 'chmod' 'clone' 'close' 'connect' 'epoll_create1'
 'epoll_ctl' 'epoll_wait' 'exit_group' 'fcntl' 'fstat' 'getcwd' 'getdents'
 'geteuid' 'getsockname' 'getsockopt' 'lseek' 'lstat' 'mmap' 'munmap'
 'open' 'openat' 'poll' 'procexit' 'read' 'rename' 'rt_sigaction'
 'rt_sigprocmask' 'select' 'semop' 'set_robust_list' 'setgid' 'setgroups'
 'setitimer' 'setuid' 'shutdown' 'socket' 'stat' 'switch' 'times' 'umask'
 'wait4' 'write' 'writev' 'accept' 'access' 'clone' 'close' 'exit' 'fcntl'
 'fsync' 'futex' 'getpeername' 'getsockname' 'madvise' 'mmap' 'mprotect'
 'munmap' 'poll' 'procexit' 'pwrite' 'rea

...

['apache2' 'id' 'mysqld' 'nan' 'sh' 'uname' 'wget']
['accept' 'access' 'arch_prctl' 'brk' 'chdir' 'chmod' 'clock_getres'
 'clone' 'close' 'connect' 'dup' 'epoll_create1' 'epoll_ctl' 'execve'
 'exit_group' 'fcntl' 'fdatasync' 'flock' 'fstat' 'fstatfs' 'fsync'
 'ftruncate' 'futex' 'getcwd' 'getdents' 'getegid' 'geteuid' 'getgid'
 'getgroups' 'getpeername' 'getpid' 'getppid' 'getrandom' 'getrlimit'
 'getrusage' 'getsockname' 'getsockopt' 'gettid' 'getuid' 'io_getevents'
 'io_submit' 'ioctl' 'lseek' 'lstat' 'madvise' 'mkdir' 'mmap' 'mprotect'
 'munmap' 'nanosleep' 'open' 'pipe' 'poll' 'pread' 'procexit' 'pwrite'
 'read' 'readlink' 'recvfrom' 'rename' 'rmdir' 'rt_sigaction'
 'rt_sigprocmask' 'sched_yield' 'select' 'sendmmsg' 'sendmsg' 'sendto'
 'set_robust_list' 'set_tid_address' 'setgid' 'setgroups' 'setitimer'
 'setsockopt' 'setuid' 'shutdown' 'signaldeliver' 'sigreturn' 'socket'
 'stat' 'statfs' 'switch' 'times' 'umask' 'uname' 'unlink' 'utime' 'wait4'
 'write' 'writev']
process list :
[

100%|███████████████████████████████████████| 1078/1078 [00:43<00:00, 24.84it/s]


['apache2' 'mysqld' 'nan']
['accept' 'access' 'brk' 'chdir' 'clone' 'close' 'connect' 'epoll_create1'
 'epoll_ctl' 'exit_group' 'fcntl' 'flock' 'fstat' 'fsync' 'ftruncate'
 'futex' 'getcwd' 'getdents' 'geteuid' 'getpeername' 'getpid'
 'getsockname' 'getsockopt' 'gettid' 'getuid' 'io_getevents' 'io_submit'
 'lseek' 'lstat' 'madvise' 'mmap' 'mprotect' 'munmap' 'nanosleep' 'open'
 'openat' 'poll' 'pread' 'procexit' 'pwrite' 'read' 'recvfrom'
 'rt_sigaction' 'rt_sigprocmask' 'sched_yield' 'select' 'sendto'
 'set_robust_list' 'setgid' 'setgroups' 'setitimer' 'setsockopt' 'setuid'
 'shutdown' 'socket' 'stat' 'switch' 'times' 'unlink' 'utime' 'wait4'
 'write' 'writev']
process list :
['apache2' 'nan' 'mysqld' 'mysqld_safe' 'nan' 'apache2' 'nan' 'nginx'
 'puma' 'reactor.rb:249' 'ruby-timer-thr' 'server.rb:358'
 'thread_pool.rb*' 'tmp_restart.rb*' 'connection_poo*' 'puma'
 'reactor.rb:249' 'ruby-timer-thr' 'server.rb:358' 'thread_pool.rb*'
 'tmp_restart.rb*' 'gs' 'nan' 'pstoedit' 'python3' 'sh'

100%|███████████████████████████████████████| 1063/1063 [06:31<00:00,  2.72it/s]

['C1' 'C2' 'Common-Cleaner' 'Finalizer' 'G1' 'GC' 'Reference' 'Service'
 'Signal' 'Sweeper' 'VM' 'java' 'nan' 'python3']
['accept' 'access' 'arch_prctl' 'brk' 'clock_getres' 'clone' 'close'
 'connect' 'execve' 'exit' 'exit_group' 'fchdir' 'fcntl' 'fstat'
 'ftruncate' 'futex' 'getcwd' 'getdents' 'getdents64' 'geteuid'
 'getrlimit' 'getrusage' 'gettid' 'getuid' 'ioctl' 'kill' 'lseek' 'lstat'
 'madvise' 'mkdir' 'mmap' 'mprotect' 'munmap' 'nanosleep' 'open' 'pipe'
 'poll' 'prctl' 'pread' 'procexit' 'read' 'readlink' 'recvfrom'
 'rt_sigaction' 'rt_sigprocmask' 'sched_getaffinity' 'sched_yield'
 'sendto' 'set_robust_list' 'set_tid_address' 'setrlimit' 'shutdown'
 'signaldeliver' 'sigreturn' 'socket' 'stat' 'switch' 'sysinfo' 'uname'
 'unlink' 'wait4' 'write']
process list :
['apache2' 'nan' 'mysqld' 'mysqld_safe' 'nan' 'apache2' 'nan' 'nginx'
 'puma' 'reactor.rb:249' 'ruby-timer-thr' 'server.rb:358'
 'thread_pool.rb*' 'tmp_restart.rb*' 'connection_poo*' 'puma'
 'reactor.rb:249' 'ruby-timer-t

In [63]:
process_arr = np.delete(process_arr,np.where(process_arr=="nan"))
event_type_arr = np.delete(event_type_arr,np.where(event_type_arr == "nan"))
print(process_arr)
print(len(process_arr)) #30 process type exist
print(event_type_arr)
print(len(event_type_arr)) # 106 process type exist

['C1' 'C2' 'Common-Cleaner' 'Finalizer' 'G1' 'GC' 'Reference' 'Service'
 'Signal' 'Sweeper' 'VM' 'apache2' 'connection_poo*' 'gs' 'id' 'java'
 'mysqld' 'mysqld_safe' 'nginx' 'pstoedit' 'puma' 'python3'
 'reactor.rb:249' 'ruby-timer-thr' 'server.rb:358' 'sh' 'thread_pool.rb*'
 'tmp_restart.rb*' 'uname' 'wget']
30
['accept' 'access' 'arch_prctl' 'brk' 'chdir' 'chmod' 'clock_getres'
 'clone' 'close' 'connect' 'dup' 'epoll_create1' 'epoll_ctl' 'epoll_wait'
 'execve' 'exit' 'exit_group' 'fchdir' 'fcntl' 'fdatasync' 'flock' 'fstat'
 'fstatfs' 'fsync' 'ftruncate' 'futex' 'getcwd' 'getdents' 'getdents64'
 'getegid' 'geteuid' 'getgid' 'getgroups' 'getpeername' 'getpid' 'getppid'
 'getrandom' 'getrlimit' 'getrusage' 'getsockname' 'getsockopt' 'gettid'
 'getuid' 'io_getevents' 'io_submit' 'ioctl' 'kill' 'lseek' 'lstat'
 'madvise' 'mkdir' 'mmap' 'mprotect' 'mremap' 'munmap' 'nanosleep'
 'newfstatat' 'open' 'openat' 'pipe' 'poll' 'prctl' 'pread' 'procexit'
 'pwrite' 'read' 'readlink' 'recvfrom' 're

process_arr을 바탕으로 process column label encoding
event_type_arr을 바탕으로 event_type column label encoding
event_direction은 one-hot encoding oor label encoding
event_num, event_time column drop
결론적으로 "cpu", "user_id", "process", "process_id", "event_direction", "event_type", "event_argument" 7개의 특성을 사용
Encoding 전에 결측치가 존재하는 행은 제거하고 Encoding한 후 에는 MinmaxScaler, StandardScaler 둘 중에 하나 사용하여 정규화
(Unsupervised Learning을 진행할 때에는 BatchNormalization을 사용할 것이기 때문에 딱히 정규화는 필요없을 듯?)
Event_argument...?

---------------------------------------------------------------------------------------------------------------
Code test part

In [452]:
run_file = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+'Bruteforce_CWE-307'+'/run/runs.csv')

In [403]:
sample = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+'ex'+'/tiny_leakey_3874.csv', index_col = 0)

In [510]:
sample = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+'ex'+'/tiny_leakey_3874.csv', index_col = 0)
sample['event_time'] = sample['event_time'].str.slice(0,8)
sample['label']=0 # 
if run_file['exploit_start_time'][run_file['scenario_name']=='tiny_leakey_3874'].values[0] != -1:
    record_start = int(sample['event_time'][0][6:8])
    exploit_start = run_file['exploit_start_time'][run_file['scenario_name']=='tiny_leakey_3874'].values[0]
    tmp_time = record_start + exploit_start
    print(record_start)
    print(exploit_start)
    if tmp_time > 60:
        real_exploit_sec = tmp_time - 60
    else:
        real_exploit_sec = tmp_time
    print(real_exploit_sec)
    sample.loc[sample['event_time'].str.slice(6,8) == str(real_exploit_sec),'label'] = 1
    print(sample['label'].value_counts())
    sample.to_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+'ex'+'/haha.csv')
    

57
22
19
0    22735
1       82
Name: label, dtype: int64
